In [1]:
import sys
from google.colab import drive

drive.mount('/content/gdrive')

!pip install --upgrade transformers

import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import AutoModel, AutoTokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 12.6 MB/s 
     |████████████████████████████████| 7.6 MB 61.1 MB/s 
     |████████████████████████████████| 182 kB 76.0 MB/s 


In [2]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/dnabert_input.csv', sep = ',', )
df.columns = ['id', 'sequence', 'label']
df.drop(columns = ['id'])

,sequence,label
0,ATTTTT TTTTTA TTTTAC TTTACA TTACAA TACAAG ACAA...,0
1,AGAGAT GAGATA AGATAG GATAGG ATAGGT TAGGTG AGGT...,0
2,TATGTA ATGTAT TGTATA GTATAT TATATA ATATAG TATA...,0
3,AGAAAT GAAATA AAATAA AATAAT ATAATA TAATAG AATA...,0
4,CTTTAA TTTAAA TTAAAA TAAAAT AAAATT AAATTA AATT...,0
...,...,...
22592,CGACAA GACAAA ACAAAG CAAAGT AAAGTT AAGTTT AGTT...,1
22593,CATATC ATATCT TATCTA ATCTAC TCTACA CTACAT TACA...,1
22594,ATACCG TACCGC ACCGCG CCGCGG CGCGGA GCGGAA CGGA...,1
22595,ATTATT TTATTC TATTCC ATTCCG TTCCGA TCCGAA CCGA...,1


In [7]:
from sklearn.model_selection import train_test_split

data = df

train_data, val_data = train_test_split(
    data, 
    test_size=0.1, 
    random_state=42,
    stratify=data['label']
)
val_data, test_data = train_test_split(
    train_data, 
    test_size=0.5, 
    random_state=42,
    stratify=train_data['label']
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

max_seq_len = 512

def convert_examples_to_features(sent_list, max_seq_len, tokenizer):

    input_ids, attention_masks, token_type_ids = [], [], []

    for sent in tqdm(sent_list, total=len(sent_list)):
        encoding_result = tokenizer.encode_plus(sent, max_length=max_seq_len, pad_to_max_length=True)

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    return (input_ids, attention_masks, token_type_ids)

X_train = convert_examples_to_features(train_data['sequence'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_val = convert_examples_to_features(val_data['sequence'], max_seq_len=max_seq_len, tokenizer=tokenizer)
X_test = convert_examples_to_features(test_data['sequence'], max_seq_len=max_seq_len, tokenizer=tokenizer)

# y_train = train_data['label'].tolist()
# y_val = val_data['label'].tolist()
# y_test = test_data['label'].tolist()

y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

# idx_encode = preprocessing.LabelEncoder()
# idx_encode.fit(train_label)

# y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
# y_val = idx_encode.transform(val_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

# label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
# idx_label = {value: key for key, value in label_idx.items()}
# print(label_idx)
# print(idx_label)

  0%|          | 0/20337 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 10169/10169 [00:07<00:00, 1306.87it/s]


In [15]:
y_train = train_data['label']
y_val = val_data['label']
y_test = test_data['label']

In [13]:
class TFAutoModelForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFAutoModelForSequenceClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [17]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# with strategy.scope():
#   model = TFAutoModelForSequenceClassification("armheb/DNA_bert_6", num_labels=2)
#   optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
#   loss = tf.keras.losses.SparseCategoricalCrossentropy()
#   model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

model = TFAutoModelForSequenceClassification("armheb/DNA_bert_6", num_labels=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])


early_stopping = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001,
    patience=2)

history = model.fit(
    X_train, y_train, epochs=5, batch_size=32, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=1024)[1]))


Some weights of the model checkpoint at armheb/DNA_bert_6 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/5


TypeError: ignored

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.plot(epochs, history.history['accuracy'])
plt.plot(epochs, history.history['val_accuracy'])
plt.title('model loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss', 'train_acc', 'val_acc'], loc='upper left')
plt.show()